# How to break your code using Python tuples

#### This notebook is a showcase (with a Genetic Algorithm as example) of how Python objects shouldn't be used.

First we have our Population class with some defined methods:
* Class constructor
* Population sorting (greater fitness first)
* Selecting the best couple (parents) for crossover operation

In [134]:
import random
from copy import deepcopy

class Population:
    def __init__(self):
        # population: list of chromosome data [data1, data2, ..., dataN]
        # data: tuple of (fitnes, chromosome)
        # chromosome: list of genes [gene1, gene2, ..., geneN]
        size = 10
        self.population = [(random.randint(1,10), range(random.randint(0,2),random.randint(3,6))) for i in range(size)]
    
    def sort(self):
        # Sorting (decreasing)
        self.population = sorted(self.population, reverse=True)
    
    def select_parents(self):
        # Actually selecting the best couple
        return [self.population[0], self.population[1]]

Then we can create and sort it just to verify which is the best couple

In [135]:
pop = Population()
print 'Before sorting'
print '\n'.join([str(ch) for ch in pop.population])
print 

pop.sort()
print 'After sorting'
print '\n'.join([str(ch) for ch in pop.population])

Before sorting
(6, [1, 2, 3, 4])
(3, [0, 1, 2])
(3, [1, 2, 3, 4, 5])
(2, [0, 1, 2, 3, 4])
(8, [0, 1, 2])
(8, [2, 3, 4])
(3, [2, 3])
(7, [2, 3, 4, 5])
(8, [0, 1, 2])
(1, [0, 1, 2, 3, 4])

After sorting
(8, [2, 3, 4])
(8, [0, 1, 2])
(8, [0, 1, 2])
(7, [2, 3, 4, 5])
(6, [1, 2, 3, 4])
(3, [2, 3])
(3, [1, 2, 3, 4, 5])
(3, [0, 1, 2])
(2, [0, 1, 2, 3, 4])
(1, [0, 1, 2, 3, 4])


Here we create a simulation of a crossover, just to switch data between parents.

What happens here is that when you call `pop.select_parents()` it returns a list containing 2 **pointers** to the objects (tuples) representing parents. Each one of these parents also store a **pointer** to the object (list) representing the genes.

This is important to remember because when we apply changes to the object `parents` we transfer information between its lists, **not creating** new ones. So, what we are really doing is modifying the list objects (storing the genes) **pointed by** each `parents` tuple.

But... can you figure out *what is also changed*?

**The objects pointed by `pop.population`!**

They're all the same in the memory, only with different tags and this will change your best individual's (in a GA) genes under the blanket.

One possible solution is to use Python's `copy.deepcopy()` to create a new tuple object so we don't mess up with the original one.

In [136]:
def apply_change_on_parent((parent1, parent2)):
        ch1, ch2 = parent1[1], parent2[1]
        # If we do the following it works without deepcopy
        # ch1, ch2 = ch2, ch1
        ch1[:], ch2[:] = ch2[:], ch1[:]
        return [(0, ch1), (0, ch2)]

# Switch comments:
#parents = deepcopy(pop.select_parents()) # parents is a whole new object that looks like the returned one
parents = pop.select_parents() # parents will point to the same object(s) pointed by the returned tuple
print 'Before change'
print parents

print

childs = apply_change_on_parent(parents)
print 'After change'
print pop.select_parents(), childs



Before change
[(8, [2, 3, 4]), (8, [0, 1, 2])]

After change
[(8, [0, 1, 2]), (8, [2, 3, 4])] [(0, [0, 1, 2]), (0, [2, 3, 4])]
